In [3]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import json
import torch
from torch.utils.data import Dataset
import os
from datetime import datetime

class KnowledgeBaseDataset(Dataset):
    def __init__(self, knowledge_base, tokenizer, max_length=512):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Processa a base de conhecimento em formato de exemplos de treino
        for category, qa_pairs in knowledge_base["perguntas_respostas"].items():
            if isinstance(qa_pairs, dict):
                # Formata cada par de pergunta/resposta como um exemplo de treino
                question = qa_pairs.get("pergunta", "")
                answer = self._format_answer(qa_pairs.get("resposta", {}))
                
                if question and answer:
                    self.examples.append(self._format_example(question, answer))
                
                # Se houver exemplos de código, adiciona como exemplos separados
                if "exemplos" in qa_pairs:
                    for context, code in qa_pairs["exemplos"].items():
                        question = f"Mostre um exemplo de código para {context}"
                        self.examples.append(self._format_example(question, code))

    def _format_answer(self, answer):
        if isinstance(answer, dict):
            # Formata respostas estruturadas
            formatted = []
            for key, value in answer.items():
                if isinstance(value, list):
                    formatted.append(f"{key}:\n" + "\n".join(f"- {item}" for item in value))
                else:
                    formatted.append(f"{key}: {value}")
            return "\n".join(formatted)
        return str(answer)

    def _format_example(self, question, answer):
        # Formato do prompt: pergunta e resposta com marcadores específicos
        prompt = f"""###Pergunta
{question}

###Resposta
{answer}

###Fim"""
        return prompt

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        # Tokeniza o exemplo com padding e truncamento
        encodings = self.tokenizer(
            self.examples[idx],
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        return {
            "input_ids": encodings["input_ids"].squeeze(),
            "attention_mask": encodings["attention_mask"].squeeze()
        }

class KnowledgeManager:
    def __init__(self, base_path="knowledge_base"):
        self.base_path = base_path
        self.knowledge_base = self._load_knowledge()
        os.makedirs(base_path, exist_ok=True)

    def _load_knowledge(self):
        # Carrega todos os arquivos de conhecimento
        knowledge = {"perguntas_respostas": {}}
        for filename in os.listdir(self.base_path):
            if filename.endswith('.json'):
                with open(os.path.join(self.base_path, filename), 'r') as f:
                    domain_knowledge = json.load(f)
                    knowledge["perguntas_respostas"].update(domain_knowledge["perguntas_respostas"])
        return knowledge

    def add_knowledge(self, domain_name, new_knowledge):
        # Adiciona novo conhecimento e salva em arquivo separado
        filename = f"{domain_name}_{datetime.now().strftime('%Y%m%d')}.json"
        filepath = os.path.join(self.base_path, filename)
        
        with open(filepath, 'w') as f:
            json.dump(new_knowledge, f, indent=2)
        
        # Atualiza a base de conhecimento em memória
        self.knowledge_base["perguntas_respostas"].update(new_knowledge["perguntas_respostas"])

def train_model(model_name, knowledge_base, output_dir="trained_model"):
    """Treina ou continua o treinamento do modelo com nova base de conhecimento."""
    # Carrega modelo e tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_name, return_dict=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Prepara dataset
    dataset = KnowledgeBaseDataset(knowledge_base, tokenizer)
    
    # Configurações de treinamento
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        save_steps=100,
        save_total_limit=2,
        logging_dir=f"{output_dir}/logs",
        learning_rate=2e-5,
    )
    
    # Inicializa trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )
    
    # Treina o modelo
    trainer.train()
    
    # Salva o modelo e tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    return model, tokenizer

# Exemplo de uso
def main():
    # Inicializa o gerenciador de conhecimento
    knowledge_manager = KnowledgeManager()
    
    # Adiciona conhecimento inicial (estagiário)
    calculo_knowledge = {
"perguntas_respostas": {
    "geral": {
      "pergunta": "O que é o objeto calculo e qual sua função principal?",
    }
}
}
    knowledge_manager.add_knowledge("estagiario", estagiario_knowledge)
    
    # Treina o modelo com todo o conhecimento
    model_name = "meta-llama/Llama-3.2-3B-Instruct"
    model, tokenizer = train_model(model_name, knowledge_manager.knowledge_base)
    
    return model, tokenizer

if __name__ == "__main__":
    model, tokenizer = main()

NameError: name 'estagiario_knowledge' is not defined